# LSTM RNN

In [1]:
import numpy as np

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear
from torch.nn.functional import softmax, relu, tanh
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

from sklearn.manifold import TSNE

from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
use_cuda = torch.cuda.is_available()

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

In [3]:
TEXT = data.Field(sequential=True, include_lengths = True)
LABEL = data.Field(sequential=False)

train_set, validation_set, test_set = datasets.SST.splits(TEXT,
                                                          LABEL,
                                                          fine_grained=False,
                                                          train_subtrees=True,
                                                          filter_pred=lambda ex: ex.label != 'neutral')

In [4]:
#for val_batch in val_iter:
#    print(val_batch.text.size())

In [5]:
# build the vocabulary
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.build_vocab(train_set, max_size=None, vectors=Vectors('wiki.simple.vec', url=url))
LABEL.build_vocab(train_set)
# print vocab information
print('len(TEXT.vocab)', len(TEXT.vocab))
print('TEXT.vocab.vectors.size()', TEXT.vocab.vectors.size())

# make iterator for splits
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_set, validation_set, test_set), batch_size=3)




# print batch information
batchsst = next(iter(train_iter))


len(TEXT.vocab) 18005
TEXT.vocab.vectors.size() torch.Size([18005, 300])


In [6]:
print(TEXT)

In [11]:
embedding_dim = TEXT.vocab.vectors.size()[1]
num_embeddings = TEXT.vocab.vectors.size()[0]
num_classes = len(LABEL.vocab.itos)
dropout_rate = 0.2

input_dim = 100

con_dim = 200

# build the LSTM model
class LSTMNet(nn.Module):

    def __init__(self):
        super(LSTMNet, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings, embedding_dim)
        # use pretrained embeddings
        self.embeddings.weight.data.copy_(TEXT.vocab.vectors)
        
        # add hidden layers
        # YOUR CODE HERE!
        
        self.lstm_input = nn.LSTM(input_size = embedding_dim,
                                  hidden_size = embedding_dim,
                                  num_layers = 1)
        
        self.input = Linear(in_features = embedding_dim,
                             out_features = input_dim,
                             bias = False)
    
        self.l_1 = Linear(in_features=con_dim,
                           out_features=con_dim,
                           bias = False)
        self.l_2 = Linear(in_features=con_dim,
                           out_features=con_dim,
                           bias=False)
        self.l_3 = Linear(in_features=con_dim,
                           out_features = con_dim,
                           bias = False)
        
        
        self.drop = nn.Dropout(p = dropout_rate)
        
        # output layer
        self.l_out = Linear(in_features=con_dim,
                            out_features=num_classes,
                            bias=False)
        
    def forward(self, x, y):
        out = {}
        x_text = x[0] #Text of input
        y_text = y[0]
        x_len = x[1] #Sequence length of input
        y_len = y[1]
       # packed_x = x
       # packed_y = y
       # print(x)
       #print(x_text.size())
        
        x_len, perm_idx1 = torch.sort(x_len,0,descending=True)
        
        y_len,perm_idx2 = torch.sort(y_len,0,descending=True)
       # print(y_len)
        #print(perm_idx2)
       # print(x_text)
       # print(x_text)
      #  print(x_text[perm_idx1])
        
        # get embeddings
        x = self.embeddings(x_text) # (bs,len,300)
        #num_len = x.size()[1]
        y = self.embeddings(y_text)
       # print(x.size())
       #print(perm_idx1)
       # ordered_x = sorted(x_text, key=len, reverse=True)
       # ordered_y = sorted(y_text, key=len, reverse=True)
        #print(ordered_x[perm_idx1.tolist()])
        x = self.drop(x)
        y = self.drop(y)
       # print(x_len.tolist())
       # print(x[perm_idx1])
        
      #  packed_x = pack(x[perm_idx1],x_len.tolist())
     #   packed_y = pack(y[perm_idx2],y_len.tolist())
        
    #    packed_x = self.embeddings(packed_x)
        
       # print(packed_x)
        
        x, (xht,xct) = self.lstm_input(x)
        y, (yht,yct) = self.lstm_input(y)
       # print(x)
        #unpacked_x, _ = unpack(x)
        #unpacked_y, _ = unpack(y)
        #xht = xht[0].cuda()
        #yht = yht[0].cuda()
       # print(xht.size())
       # print(unpacked_x.size())
       # print(yht[0].size())
        
        xt = xht
       #print(xt)
        yt = yht
        #print(x.size())
        #print(xt.size())
        #tanh # (bs,100)
        
        xt = torch.tanh(self.input(xt))
        yt = torch.tanh(self.input(yt))
        
        #print(xt.size())
 
        z = torch.cat((xt,yt),2)
    
        #print(z.size())
    
        z = torch.squeeze(z,0)
        
        #print(z.size())
                
        z = torch.tanh(self.l_1(z))     
        z = torch.tanh(self.l_2(z))
        z = torch.tanh(self.l_3(z))
        #print(z.size())
        
        
        # Softmax
        out['out'] = self.l_out(z)
        return out

net = LSTMNet()
if use_cuda:
    net.cuda()
print(net)

LSTMNet(
  (embeddings): Embedding(18005, 300)
  (lstm_input): LSTM(300, 300)
  (input): Linear(in_features=300, out_features=100, bias=False)
  (l_1): Linear(in_features=200, out_features=200, bias=False)
  (l_2): Linear(in_features=200, out_features=200, bias=False)
  (l_3): Linear(in_features=200, out_features=200, bias=False)
  (drop): Dropout(p=0.2)
  (l_out): Linear(in_features=200, out_features=3, bias=False)
)


In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(net.parameters(), lr=0.01,weight_decay=0.001)

def accuracy(ys, ts):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = torch.eq(torch.max(ys, 1)[1], ts)
    # averaging the one-hot encoded vector
    return torch.mean(correct_prediction.float())

In [16]:
def construct_sentences(batch):
    """    
    Parameters
    ----------
    batch: torchtext.data.batch.Batch
    
    Returns
    -------
    [str]
    """
    return [" ".join([TEXT.vocab.itos[elm] 
                      for elm in get_numpy(batch.text[:,i])])
            for i in range(batch.text.size()[1])]

def get_labels(batch):
    """
    Parameters
    ----------
    batch: torchtext.data.batch.Batch
    
    Returns
    -------
    [str]
    """
    return [LABEL.vocab.itos[get_numpy(batch.label[i])] for i in range(len(batch.label))]


In [17]:
max_iter = 1000
eval_every = 500
log_every = 200

# will be updated while iterating
#tsne_plot = show(p, notebook_handle=True)

train_loss, train_accs = [], []

net.train()
for i, batch in enumerate(train_iter):
    if i % eval_every == 0:
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0
        val_meta = {'label_idx': [], 'sentences': [], 'labels': []}
        for val_batch in val_iter:
            #print(val_batch.text[0][0])
            output = net(val_batch.text,val_batch.text)
            # batches sizes might vary, which is why we cannot just mean the batch's loss
            # we multiply the loss and accuracies with the batch's size,
            # to later divide by the total size
            #print(val_batch.text.size())
            #print(val_batch.label.size())
            #print(output['out'].size())
            val_losses += criterion(output['out'], val_batch.label) * val_batch.batch_size
            val_accs += accuracy(output['out'], val_batch.label) * val_batch.batch_size
            val_lengths += val_batch.batch_size
            
        
        # divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        
        print("valid, it: {} loss: {:.2f} accs: {:.2f}\n".format(i, get_numpy(val_losses), get_numpy(val_accs)))
        #update_plot(val_meta, 'LSTM', tsne_plot)
        
        net.train()
    
    output = net(batch.text,batch.text)
    batch_loss = criterion(output['out'], batch.label)
    
    train_loss.append(get_numpy(batch_loss))
    train_accs.append(get_numpy(accuracy(output['out'], batch.label)))
    
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    
    if i % log_every == 0:        
        print("train, it: {} loss: {:.2f} accs: {:.2f}".format(i, 
                                                               np.mean(train_loss), 
                                                               np.mean(train_accs)))
        # reset
        train_loss, train_accs = [], []
        
    if max_iter < i:
        break

C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train), lengths
C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


valid, it: 0 loss: 1.10 accs: 0.27

train, it: 0 loss: 1.10 accs: 0.33
train, it: 200 loss: 1.10 accs: 0.55
train, it: 400 loss: 1.09 accs: 0.57
valid, it: 500 loss: 1.08 accs: 0.51

train, it: 600 loss: 1.08 accs: 0.61
train, it: 800 loss: 1.05 accs: 0.56
valid, it: 1000 loss: 0.77 accs: 0.51

train, it: 1000 loss: 0.89 accs: 0.56


# SNLI

In [18]:
print("Run test on SNLI...")
TEXT = datasets.snli.ParsedTextField()
LABEL = data.LabelField()
TREE = datasets.snli.ShiftReduceField()

train, val, test = datasets.SNLI.splits(TEXT, LABEL, TREE)

print("Fields:", train.fields)
print("Number of examples:\n", len(train))
print("First Example instance:\n", vars(train[0]))

url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.build_vocab(train, max_size=None, vectors=[CharNGram(),Vectors('wiki.simple.vec', url=url)])
#TEXT.build_vocab(train,vectors=GloVe[name='840B',dim='300'])
LABEL.build_vocab(train)


train_iter, val_iter, test_iter = data.Iterator.splits((train, val, test), batch_size=3)

#batch = next(iter(train_iter))
#print("Numericalize premises:\n", batch.premise)
#print("Numericalize hypotheses:\n", batch.hypothesis)
#print("Entailment labels:\n", batch.label)

print("Test iters function")
train_iter, val_iter, test_iter = datasets.SNLI.iters(batch_size=4, trees=True)

batch = next(iter(train_iter))
print("Numericalize premises:\n", batch.premise)
print("Numericalize hypotheses:\n", batch.hypothesis)
print("Entailment labels:\n", batch.label)

#val_iter_set = next(iter(val_iter))


Run test on SNLI...
Fields: {'premise': <torchtext.datasets.snli.ParsedTextField object at 0x0000026AA47BEF28>, 'premise_transitions': <torchtext.datasets.snli.ShiftReduceField object at 0x0000026AA47BE160>, 'hypothesis': <torchtext.datasets.snli.ParsedTextField object at 0x0000026AA47BEF28>, 'hypothesis_transitions': <torchtext.datasets.snli.ShiftReduceField object at 0x0000026AA47BE160>, 'label': <torchtext.data.field.LabelField object at 0x0000026AA47BE0F0>}
Number of examples:
 549367
First Example instance:
 {'premise': ['A', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.'], 'premise_transitions': ['shift', 'shift', 'reduce', 'shift', 'shift', 'shift', 'reduce', 'reduce', 'reduce', 'shift', 'shift', 'shift', 'shift', 'shift', 'shift', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'shift', 'reduce', 'reduce'], 'hypothesis': ['A', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.'], 'hypothesis_transitions': ['shift',

In [19]:
embedding_dim = TEXT.vocab.vectors.size()[1]
num_embeddings = TEXT.vocab.vectors.size()[0]
num_classes = len(LABEL.vocab.itos)
dropout_rate = 0.2

input_dim = 100

con_dim = 200

# build the LSTM model
class LSTMNet(nn.Module):

    def __init__(self):
        super(LSTMNet, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings, embedding_dim)
        # use pretrained embeddings
        self.embeddings.weight.data.copy_(TEXT.vocab.vectors)
        
        # add hidden layers
        # YOUR CODE HERE!
        
        self.lstm_input = nn.LSTM(input_size = embedding_dim,
                                  hidden_size = embedding_dim,
                                  batch_first = False,
                                  num_layers = 1)
        
        self.input = Linear(in_features = embedding_dim,
                             out_features = input_dim,
                             bias = False)
    
        self.l_1 = Linear(in_features=con_dim,
                           out_features=con_dim,
                           bias = False)
        self.l_2 = Linear(in_features=con_dim,
                           out_features=con_dim,
                           bias=False)
        self.l_3 = Linear(in_features=con_dim,
                           out_features = con_dim,
                           bias = False)
        
        
        self.drop = nn.Dropout(p = dropout_rate)
        
        # output layer
        self.l_out = Linear(in_features=con_dim,
                            out_features=num_classes,
                            bias=False)
        
    def forward(self, x, y):
        out = {}
        # get embeddings
        x = self.embeddings(x) # (bs,len,300)
        #num_len = x.size()[1]
        y = self.embeddings(y)
        
        x = self.drop(x)
        y = self.drop(y)
        
        x, xht = self.lstm_input(x)
        y, yht = self.lstm_input(y)
        
        xt = xht[0]
        yt = yht[0]
        #print(x.size())
        #print(xt.size())
        #tanh # (bs,100)
        
        xt = torch.tanh(self.input(xt))
        yt = torch.tanh(self.input(yt))
 
        z = torch.cat((xt,yt),2)
    
        z = torch.squeeze(z,0)
                
        z = torch.tanh(self.l_1(z))     
        z = torch.tanh(self.l_2(z))
        z = torch.tanh(self.l_3(z))
      #  print(z.size())
        
        
        # Softmax
        out['out'] = self.l_out(z)
        return out

net = LSTMNet()
if use_cuda:
    net.cuda()
print(net)

LSTMNet(
  (embeddings): Embedding(42392, 400)
  (lstm_input): LSTM(400, 400)
  (input): Linear(in_features=400, out_features=100, bias=False)
  (l_1): Linear(in_features=200, out_features=200, bias=False)
  (l_2): Linear(in_features=200, out_features=200, bias=False)
  (l_3): Linear(in_features=200, out_features=200, bias=False)
  (drop): Dropout(p=0.2)
  (l_out): Linear(in_features=200, out_features=3, bias=False)
)


In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(net.parameters(), lr=0.01,weight_decay=0.001)

def accuracy(ys, ts):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = torch.eq(torch.max(ys, 1)[1], ts)
    # averaging the one-hot encoded vector
    return torch.mean(correct_prediction.float())

In [22]:
max_iter = 2000
eval_every = 500
log_every = 200

# will be updated while iterating
#tsne_plot = show(p, notebook_handle=True)

train_loss, train_accs = [], []

net.train()
for i, batch in enumerate(train_iter):
    if i % eval_every == 0:
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0
       # val_meta = {'label_idx': [], 'sentences': [], 'labels': []}
        for val_batch in val_iter:
            output = net(val_batch.premise,val_batch.hypothesis)
            # batches sizes might vary, which is why we cannot just mean the batch's loss
            # we multiply the loss and accuracies with the batch's size,
            # to later divide by the total size
            #print(output['out'])
            #print(val_batch.label)
            val_losses += criterion(output['out'], val_batch.label) * val_batch.batch_size
            val_accs += accuracy(output['out'], val_batch.label) * val_batch.batch_size
            val_lengths += val_batch.batch_size
           # print(val_batch.batch_size)
            
        
        # divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        
        print("valid, it: {} loss: {:.2f} accs: {:.2f}\n".format(i, get_numpy(val_losses), get_numpy(val_accs)))
        #update_plot(val_meta, 'bow', tsne_plot)
        
        net.train()
    
    output = net(batch.premise,batch.hypothesis)
    batch_loss = criterion(output['out'], batch.label)
    
    train_loss.append(get_numpy(batch_loss))
    train_accs.append(get_numpy(accuracy(output['out'], batch.label)))
    
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    
    if i % log_every == 0:        
        print("train, it: {} loss: {:.2f} accs: {:.2f}".format(i, 
                                                               np.mean(train_loss), 
                                                               np.mean(train_accs)))
        # reset
        train_loss, train_accs = [], []
        
    if max_iter < i:
        break

C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


valid, it: 0 loss: 1.01 accs: 0.27



RuntimeError: CUDA error: out of memory